In [1]:
from pyhive import hive
import pymysql
conn = hive.Connection(host='127.0.0.1', port=10000, username='hadoop')
cursor = conn.cursor()

In [2]:
def exec_sql(sql):
    cursor.execute(sql)
    for result in cursor.fetchall():
        print(result)

# 对缺失项进行预处理

# 实现hive和hbase映射

In [3]:
cursor.execute("DROP TABLE IF EXISTS film44")

In [4]:
mapping_from_hbase_to_hive_sql = '''CREATE EXTERNAL TABLE film44(id STRING, 
name string, 
male float,
female float,
under20 float, 
20to24 float,
25to29 float, 
30to34 float, 
35to39 float, 
above40 float, 
degree_high float, 
degree_low float, 
city1 float, 
city2 float, 
city3 float, 
city4 float, 
occupation_whitecollar float, 
occupation_student float, 
occupation_others float, 
preference_action float, 
preference_comedy float, 
preference_romance float, 
occupation_science float, 
occupation_animate float) STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler' 
WITH SERDEPROPERTIES ("hbase.columns.mapping" = ":key,
f:n,
f:m,
f:fm,
f:un20,
f:20to24,
f:25to29,
f:30to34,
f:35to39,
f:ab40,
f:dg_h,
f:dg_l,
f:ct1,
f:ct2,
f:ct3,
f:ct4,
f:oc_w,
f:oc_s,
f:oc_o,
f:pf_ac,
f:pf_co,
f:pf_ro,
f:pf_sc,
f:pf_an") TBLPROPERTIES("hbase.table.name" = "film22")'''

In [5]:
cursor.execute(mapping_from_hbase_to_hive_sql)

In [6]:
exec_sql('select * from film44 limit 3')

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.384, 0.616, 0.052, 0.27, 0.263, 0.174, 0.134, 0.107, 0.248, 0.752, 0.169, 0.443, 0.18, 0.208, 0.5, 0.19, 0.31, 0.46, 0.12, 0.16, 0.09, 0.09)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.374, 0.626, 0.087, 0.352, 0.258, 0.124, 0.086, 0.092, 0.32, 0.68, 0.12, 0.405, 0.202, 0.272, 0.39, 0.25, 0.35, 0.45, 0.14, 0.18, 0.07, 0.06)
('0075bdf21f40e7cd96bc973028282ede', '梭哈人生', 0.445, 0.555, 0.09, 0.335, 0.242, 0.125, 0.098, 0.11, 0.308, 0.692, 0.156, 0.439, 0.181, 0.223, 0.39, 0.27, 0.34, 0.46, 0.13, 0.18, 0.08, 0.06)


In [7]:
exec_sql('select count(*) from film44')

(878,)


# after_prep（进行第一遍筛选，将带NULL值行全部删除）

In [8]:
cursor.execute("DROP TABLE IF EXISTS film44_after_prep")

In [9]:
#建表用于存放筛选后的数据
create_first_prep_sql= '''create table film44_after_prep(id string,
name string,
male float,
female float,
under20 float,
20to24 float,
25to29 float,
30to34 float,
35to39 float,
above40 float,
degree_high float,
degree_low float,
city1 float,
city2 float,
city3 float,
city4 float,
occupation_whitecollar float,
occupation_student float,
occupation_others float,
preference_action float,
preference_comedy float,
preference_romance float,
occupation_science float,
occupation_animate float) ROW format delimited fields terminated by ',' STORED AS TEXTFILE'''

In [10]:
cursor.execute(create_first_prep_sql)

In [11]:
#将筛选后的数据进行插入操作
insert_table_sql = '''insert overwrite table film44_after_prep select * from film44 where id IS NOT NULL AND name IS NOT NULL AND male IS NOT NULL AND female IS NOT NULL AND under20 IS NOT NULL AND 20to24 IS NOT NULL AND 25to29 IS NOT NULL AND 30to34 IS NOT NULL AND 35to39 IS NOT NULL AND above40 IS NOT NULL AND degree_high IS NOT NULL AND degree_low IS NOT NULL AND city1 IS NOT NULL AND city2 IS NOT NULL AND city3 IS NOT NULL AND city4 IS NOT NULL AND occupation_whitecollar IS NOT NULL AND occupation_student IS NOT NULL AND occupation_others IS NOT NULL AND occupation_animate IS NOT NULL '''

In [12]:
cursor.execute(insert_table_sql)

In [13]:
count_sql = 'select count(*) from film44'
exec_sql(count_sql)

(878,)


In [14]:
sql = 'select * from film44_after_prep limit 3'
exec_sql(sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.384, 0.616, 0.052, 0.27, 0.263, 0.174, 0.134, 0.107, 0.248, 0.752, 0.169, 0.443, 0.18, 0.208, 0.5, 0.19, 0.31, 0.46, 0.12, 0.16, 0.09, 0.09)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.374, 0.626, 0.087, 0.352, 0.258, 0.124, 0.086, 0.092, 0.32, 0.68, 0.12, 0.405, 0.202, 0.272, 0.39, 0.25, 0.35, 0.45, 0.14, 0.18, 0.07, 0.06)
('0075bdf21f40e7cd96bc973028282ede', '梭哈人生', 0.445, 0.555, 0.09, 0.335, 0.242, 0.125, 0.098, 0.11, 0.308, 0.692, 0.156, 0.439, 0.181, 0.223, 0.39, 0.27, 0.34, 0.46, 0.13, 0.18, 0.08, 0.06)


# 依据需求文档进行数据提取

# table_1_PUPsex

In [29]:
#piao,data待补充
PUPsex_sql = 'select id,name,male,female from film44_after_prep limit 2'
exec_sql(PUPsex_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.384, 0.616)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.374, 0.626)


# table_2_CUPsex

In [31]:
#mclass,data待补充
CUPsex_sql = 'select id,name,male,female from film44_after_prep limit 2'
exec_sql(CUPsex_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.384, 0.616)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.374, 0.626)


# table_3_PUPage

In [35]:
#data,piao待补充
PUPage_sql = 'select id,name,under20,20to24,25to29,30to34,above40 from film44_after_prep limit 2'
exec_sql(PUPage_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.052, 0.27, 0.263, 0.174, 0.107)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.087, 0.352, 0.258, 0.124, 0.092)


# table_4_CUPage

In [36]:
#data,mclass待补充
CUPage_sql = 'select id,name,under20,20to24,25to29,30to34,above40 from film44_after_prep limit 2'
exec_sql(CUPage_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.052, 0.27, 0.263, 0.174, 0.107)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.087, 0.352, 0.258, 0.124, 0.092)


# table_5_PUPedu

In [38]:
#data,piao待补充
PUPedu_sql = 'select id,name,degree_high,degree_low from film44_after_prep limit 2'
exec_sql(PUPedu_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.248, 0.752)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.32, 0.68)


# table_6_PUPdistrict

In [39]:
#data,piao待补充
PUPdistrict_sql = 'select city1,city2,city3,city4 from film44_after_prep limit 2'
exec_sql(PUPdistrict_sql)

(0.169, 0.443, 0.18, 0.208)
(0.12, 0.405, 0.202, 0.272)


# table_7_PMUjob

In [42]:
#data，piao待补充
PMUjob_sql = 'select id,name,occupation_whitecollar,occupation_student,occupation_others from film44_after_prep limit 2'
exec_sql(PMUjob_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.5, 0.19, 0.31)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.39, 0.25, 0.35)


# table_8_CMUjob

In [44]:
#data,mclass待补充
CMUjob_sql = 'select id,name,occupation_whitecollar,occupation_student,occupation_others from film44_after_prep limit 2'
exec_sql(CMUjob_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩', 0.5, 0.19, 0.31)
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店', 0.39, 0.25, 0.35)


# table_9_DataPiao

In [46]:
#data,piao待补充
DataPiao_sql = 'select id,name from film44_after_prep limit 2'
exec_sql(DataPiao_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩')
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店')


# table_10_DataClass

In [47]:
#data,mclass待补充
DataClass_sql = 'select id,name from film44_after_prep limit 2'
exec_sql(DataClass_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩')
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店')


# table_11_PiaoClass

In [48]:
#mclass,piao待补充
PiaoClass_sql = 'select id,name from film44_after_prep limit 2'
exec_sql(PiaoClass_sql)

('0019a756b48d308515f7fa27f3acbb6d', '奇迹男孩')
('00473ab28e48e27a31c111e4d25ef43f', '解忧杂货店')


# 写进数据库

# 用pandas查看一下

In [21]:
#打开数据库连接
db= pymysql.connect(host="127.0.0.1",user="root",
     password="123456",db="comex",port=3306)

In [22]:
import pandas as pd

In [23]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [24]:
sql = 'select * from personas'
pd = pd.read_sql(sql,db)

In [25]:
pd.head(5)

,id,name,male,female,under20,20to24,25to29,30to34,35to39,above40,degree_high,degree_low,city1,city2,city3,city4,occupation_whitecollar,occupation_student,occupation_others,preference_action,preference_comedy,preference_romance,occupation_science,occupation_animate
0,c2ea3dd867e9943bec89008ac3852cca,八恶人,0.696,0.304,0.086,0.326,0.283,0.131,0.067,0.107,0.332,0.668,0.138,0.491,0.14,0.231,0.41,0.13,0.46,0.32,0.11,0.27,0.18,0.05
1,c33b2c51d5cc8ebfce87dae8aac3f3f6,乌龙特工,0.708,0.292,0.106,0.33,0.268,0.117,0.075,0.104,0.267,0.733,0.141,0.472,0.169,0.218,0.38,0.11,0.51,0.32,0.11,0.27,0.18,0.05
2,c33ee9bd38bd25940cdf4b603b96a6d7,相爱相亲,0.393,0.607,0.059,0.304,0.29,0.151,0.093,0.102,0.631,0.369,0.167,0.412,0.194,0.227,0.49,0.18,0.33,0.46,0.12,0.19,0.07,0.06
3,c343fe7fd3530ef2cc3e8016046c7642,屏住呼吸,0.631,0.369,0.116,0.363,0.236,0.091,0.079,0.114,0.281,0.719,0.164,0.465,0.172,0.199,0.32,0.16,0.52,0.32,0.11,0.27,0.18,0.05
4,c3e8465a2641d7aa69aab8d45add78ba,不期而遇,0.393,0.607,0.105,0.391,0.26,0.11,0.063,0.07,0.685,0.315,0.134,0.401,0.202,0.263,0.31,0.2,0.49,0.44,0.14,0.21,0.07,0.05
